In [1]:
import numpy as np
import pandas as pd
import mne
import os
import conformer, eeginception, eegnet
from importlib import reload
import torch
from typing import Literal, Optional
from torch.utils.data import Dataset, DataLoader, random_split, WeightedRandomSampler
from torchinfo import summary

In [59]:
from torch import nn
import torch.fx
import eegwrn

# Example: Wrap your EEGWRN in torch.fx for debugging
class EEGWRNWrapper(nn.Module):
    def __init__(self, model):
        super(EEGWRNWrapper, self).__init__()
        self.model = model

    def forward(self, x):
        embeddings, outputs = self.model(x)
        return embeddings, outputs

# Instantiate and wrap your model
model = eegwrn.EEGWRN(depth=34, width=20, num_electrodes=8, num_timepoints=100)
wrapped_model = EEGWRNWrapper(model)

# Trace the model
traced_model = torch.fx.symbolic_trace(wrapped_model)

# Inspect the traced graph
print(traced_model.graph)

/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Input shape: Proxy(getattr_1)
After Initial Convolutional shape: Proxy(getattr_2)

Input shape: Proxy(getattr_3)
After BatchNorm1: Proxy(getattr_4)
After TempConv: Proxy(getattr_5)
After BatchNorm2: Proxy(getattr_6)
After SpatialConv: Proxy(getattr_7)
After SeparableConv: Proxy(getattr_8)

Input shape: Proxy(getattr_9)
After BatchNorm1: Proxy(getattr_10)
After TempConv: Proxy(getattr_11)
After BatchNorm2: Proxy(getattr_12)
After SpatialConv: Proxy(getattr_13)
After SeparableConv: Proxy(getattr_14)

Input shape: Proxy(getattr_15)
After BatchNorm1: Proxy(getattr_16)
After TempConv: Proxy(getattr_17)
After BatchNorm2: Proxy(getattr_18)
After SpatialConv: Proxy(getattr_19)
After SeparableConv: Proxy(getattr_20)

Input shape: Proxy(getattr_21)
After BatchNorm1: Proxy(getattr_22)
After TempConv: Proxy(getattr_23)
After BatchNorm2: Proxy(getattr_24)
After SpatialConv: Proxy(getattr_25)
After SeparableConv: Proxy(getattr_26)

Input shape: Proxy(getattr_27)
After BatchNorm1: Proxy(getattr_28)
A

In [60]:
def print_shape(name):
    def hook(module, input, output):
        print(f"{name} - Input shape: {input[0].shape}, Output shape: {output.shape}")
    return hook

model.conv1.register_forward_hook(print_shape("Conv1"))
model.block1[0].register_forward_hook(print_shape("Block1_0"))

In [3]:
x = torch.randn(8, 1, 8, 100)  # Example input
embeddings, outputs = model(x)
print(f"Embeddings shape: {embeddings.shape}, Outputs shape: {outputs.shape}")

NameError: name 'model' is not defined

In [11]:
from torchviz import make_dot
import torch
reload(eegwrn)

# Create an instance of your model
model = eegwrn.EEGWRN(depth=34, width=10, num_electrodes=8, num_timepoints=100)

# Generate a dummy input
x = torch.randn(8, 1, 8, 100)

# Perform a forward pass and visualize
embeddings, outputs = model(x)
dot = make_dot((embeddings, outputs), params=dict(model.named_parameters()))

# Save the graph
dot.render("EEGWRN_graph", format="png")  # This will save as EEGWRN_graph.png

TypeError: calculate_padding() takes 1 positional argument but 2 were given

In [76]:
np.mean([[13.0, 15.0, 24.0, 33.0, 34.0, 44.0, 52.0, 50.0, 54.0, 61.0, 58.0, 66.0, 71.0, 71.0, 76.0],
[27.0, 31.0, 45.0, 55.0, 59.0, 61.0, 70.0, 67.0, 74.0, 75.0, 78.0, 82.0, 86.0, 86.0, 87.0]], axis=0)

array([20. , 23. , 34.5, 44. , 46.5, 52.5, 61. , 58.5, 64. , 68. , 68. ,
       74. , 78.5, 78.5, 81.5])

In [5]:
np.randint(0, 100)

AttributeError: module 'numpy' has no attribute 'randint'

In [62]:
import torch
import torch.nn as nn
from torch.profiler import profile, record_function, ProfilerActivity
import eegwrn

reload(eegwrn)

torch.cuda.set_device(2)
torch.cuda.empty_cache()
torch.backends.cudnn.benchmark = True

# Example Input
batch_size = 512
in_channels = 1
num_electrodes = 8
num_timepoints = 100
input_tensor = torch.randn(batch_size, in_channels, num_electrodes, num_timepoints).cuda()

# Assuming the EEGWRN model is defined as provided
model = eegwrn.EEGWRN(depth=10, width=2, num_electrodes=num_electrodes, num_timepoints=num_timepoints).cuda()

# Enable profiler to record CUDA and CPU memory
with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], 
    record_shapes=True,  # To capture the shape of tensors
    with_stack=True      # To capture stack traces for better debugging
) as prof:
    with record_function("model_forward"):
        embeddings, output = model(input_tensor)

# Display profiling results sorted by CUDA memory usage
print(prof.key_averages().table(sort_by="cuda_memory_usage", row_limit=10))

Input shape: torch.Size([512, 1, 8, 100])
After Initial Convolutional shape: torch.Size([512, 16, 8, 100])

Input shape: torch.Size([512, 16, 8, 100])
After BatchNorm1: torch.Size([512, 16, 8, 100])
After TempConv: torch.Size([512, 32, 8, 100])
After BatchNorm2: torch.Size([512, 32, 8, 100])
After SpatialConv: torch.Size([512, 64, 1, 100])
After SeparableConv: torch.Size([512, 32, 1, 100])
After SE block: torch.Size([512, 32, 8, 100])

Input shape: torch.Size([512, 32, 8, 50])
After BatchNorm1: torch.Size([512, 32, 8, 50])
After TempConv: torch.Size([512, 64, 8, 50])
After BatchNorm2: torch.Size([512, 64, 8, 50])
After SpatialConv: torch.Size([512, 128, 1, 50])
After SeparableConv: torch.Size([512, 64, 1, 50])
After SE block: torch.Size([512, 64, 8, 50])

Input shape: torch.Size([512, 64, 8, 25])
After BatchNorm1: torch.Size([512, 64, 8, 25])
After TempConv: torch.Size([512, 128, 8, 25])
After BatchNorm2: torch.Size([512, 128, 8, 25])
After SpatialConv: torch.Size([512, 256, 1, 25])
Af

In [58]:
import conformer, eeginception, eegnet, eegwrn
sample_data = torch.tensor(np.random.rand(5, 1, 8, 100), dtype=torch.float32)
reload(eegwrn)
eegwrn_model = eegwrn.EEGWRN()

embs, x = eegwrn_model(sample_data)
print(embs.shape)

summary(eegwrn_model)

Input shape: torch.Size([5, 1, 8, 100])
After Initial Convolutional shape: torch.Size([5, 16, 8, 100])

Input shape: torch.Size([5, 16, 8, 100])
After BatchNorm1: torch.Size([5, 16, 8, 100])
After TempConv: torch.Size([5, 160, 8, 100])
After BatchNorm2: torch.Size([5, 160, 8, 100])
After SpatialConv: torch.Size([5, 320, 1, 100])
After SeparableConv: torch.Size([5, 160, 1, 100])
After SE block: torch.Size([5, 160, 8, 100])

Input shape: torch.Size([5, 160, 8, 100])
After BatchNorm1: torch.Size([5, 160, 8, 100])
After TempConv: torch.Size([5, 160, 8, 100])
After BatchNorm2: torch.Size([5, 160, 8, 100])
After SpatialConv: torch.Size([5, 320, 1, 100])
After SeparableConv: torch.Size([5, 160, 1, 100])
After SE block: torch.Size([5, 160, 8, 100])

Input shape: torch.Size([5, 160, 8, 50])
After BatchNorm1: torch.Size([5, 160, 8, 50])
After TempConv: torch.Size([5, 320, 8, 50])
After BatchNorm2: torch.Size([5, 320, 8, 50])
After SpatialConv: torch.Size([5, 640, 1, 50])
After SeparableConv: tor

Layer (type:depth-idx)                   Param #
EEGWRN                                   --
├─Conv2d: 1-1                            1,584
├─Sequential: 1-2                        --
│    └─WideResBlock: 2-1                 --
│    │    └─BatchNorm2d: 3-1             32
│    │    └─ELU: 3-2                     --
│    │    └─EEGBlock: 3-3                216,032
│    │    └─Conv2d: 3-4                  2,560
│    │    └─BatchNorm2d: 3-5             320
│    └─SpatialTemporalSEBlock: 2-2       --
│    │    └─AdaptiveAvgPool2d: 3-6       --
│    │    └─Linear: 3-7                  1,610
│    │    └─Linear: 3-8                  1,760
│    │    └─ReLU: 3-9                    --
│    │    └─Sigmoid: 3-10                --
│    └─WideResBlock: 2-3                 --
│    │    └─BatchNorm2d: 3-11            320
│    │    └─ELU: 3-12                    --
│    │    └─EEGBlock: 3-13               1,667,840
│    │    └─Identity: 3-14               --
│    │    └─BatchNorm2d: 3-15            320
